In [35]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from math import sqrt
import numpy as np

# Define constants
data_path = "../Data"
seed = 1
submission_path = "../Submissions"

# 
train_y = pd.read_csv(data_path + "/train_labels.csv")
train_y.set_index("patient_id", inplace=True)
val1 = train_y['heart_disease_present']

In [25]:
all_features = pd.read_csv(data_path + "/train_values.csv")
all_features.set_index("patient_id", inplace=True)

In [36]:
features1 = [
    "max_heart_rate_achieved", "age", "oldpeak_eq_st_depression"
]

X1 = all_features[features1]

train_X1, test_X1, train_y1, test_y1 = train_test_split(
    X1, val1, test_size = 0.6, random_state=1
)

model = LogisticRegression(random_state = 1)
model.fit(train_X1, train_y1)
pred1 = model.predict(test_X1)
logloss = log_loss(test_y1, pred1)
logloss

11.193158998231711

In [6]:
test_X = pd.read_csv(data_path + "/test_values.csv")
test_X.set_index("patient_id", inplace=True)


def create_submission(file_name, train_X, train_y, pred_X, pred_index):
    model = LogisticRegression(random_state = 1)
    model.fit(train_X, train_y)
    predictions = model.predict_proba(pred_X)
    submission = pd.DataFrame(predictions[:,0], index = pred_index, columns=['heart_disease_present'])
    submission.reset_index(inplace=True)
    submission.to_csv(submission_path + "/" + file_name, index=False)
    
    return model

def coefficients(model, features):
    return pd.DataFrame(model.coef_[0,:], features, columns=["coef"])

In [7]:
model1 = create_submission(
    "submission1.csv",
    all_features[features1],
    val,
    test_X[features1],
    test_X.index
)
model1.score(all_features[features1], val)

NameError: name 'val' is not defined

In [ ]:
coefficients(model1, features1)

In [ ]:
features2 = features1 + ['resting_ekg_results', 'serum_cholesterol_mg_per_dl', 'num_major_vessels', 'resting_blood_pressure']

In [ ]:
model2 = create_submission(
    "submission2.csv",
    all_features[features2],
    val,
    test_X[features2],
    test_X.index
)

model2.score(all_features[features2], val)

In [ ]:
coef2 = coefficients(model2, features2)
coef2.sort_values("coef")

In [ ]:
features3 = [x for x in features2 if x != "resting_ekg_results"]
features3

In [ ]:
model3 = create_submission(
    "submission3.csv",
    all_features[features3],
    val,
    test_X[features3],
    test_X.index
)

model2.score(all_features[features3], val)